In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle as pkl
from sklearn.model_selection import train_test_split

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from pygam.terms import s,l,f
from pygam.pygam import LogisticGAM
from sklearn.metrics import (precision_score,f1_score,recall_score,accuracy_score,make_scorer)

from time import time
from alibi.explainers import CounterFactual

#### Counterfactual Explanations

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

df = pd.read_csv("datasets/dataset_final.csv") 
gb = pkl.load(open('models/gb.pkl', 'rb'))

labels = df.iloc[:,0]
features = df.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 0)

GradientBoostingClassifier(learning_rate=0.2, max_depth=11, n_estimators=300,
                           subsample=0.8)

In [17]:
#Original instance
x_new = X_test.iloc[0]
x_new

lead_time                        84.0
arrival_date_week_number         32.0
arrival_date_day_of_month         4.0
stays_in_weekend_nights           0.0
stays_in_week_nights              2.0
                                 ... 
deposit_type_Non Refund           0.0
deposit_type_Refundable           0.0
customer_type_Group               0.0
customer_type_Transient           1.0
customer_type_Transient-Party     0.0
Name: 93981, Length: 82, dtype: float64

In [18]:
#Params
shape = (1,) + X_test.shape[1:]
target_proba = 1.0
tol = 0.2 # want counterfactuals with p(class)>0.8
target_class = 'other'
max_iter = 1000
lam_init = 1e-1
max_lam_steps = 10
learning_rate_init = 0.1
feature_range = (X_test.min(),X_test.max())

In [22]:
#Initialize Explainer
cf = CounterFactual(gb.predict_proba, shape=shape, target_proba=target_proba, tol=tol,
                    target_class=target_class, max_iter=max_iter, lam_init=lam_init,
                    max_lam_steps=max_lam_steps, learning_rate_init=learning_rate_init,
                    feature_range=feature_range)

start_time = time()
explanation = cf.explain(x_new)
print('Explanation took {:.3f} sec'.format(time() - start_time))

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [ ]:
# pred_class = explanation.cf['class']
# proba = explanation.cf['proba'][0][pred_class]

# print(f'Counterfactual prediction: {pred_class} with probability {proba}')